# Modelado, Simulación y Optimización - Proyecto 2


Amalia Cabonell - 202122079 \
Santiago Arenas - 202220359 \
Santiago Casasbuenas - 202214932

#Problema a resolver - Caso C
Escenario Complejo con Peajes y Restricciones de Peso (Caso 3):
Incorporar restricciones de peso por municipio, asociadas a normativas
locales.
Incluir peajes con tarifas variables según el peso y tramo recorrido.
Determinar estrategias conjuntas de ruteo y recarga, optimizando el
costo total nacional.

##Modelo Matematico

### Definición de conjuntos
Se especificará de forma clara los conjuntos relevantes vinculados al problema.

#### 1. Conjunto de Puntos de Acceso (Puertos)

$P$: Conjunto de nodos origen donde se ingresan las mercancias.

Ejemplo: $P = {p_1}$ donde $p_1 = $ Puerto Barranquilla

#### 2. Conjunto de Destinos (Centros de Consumo)

$D$: Conjunto de destinos finales o municipios consumidores.

Ejemplo: $D = {d_1, d_2, d_3}$ donde:

$d_1 = $ Bogotá

$d_2 = $ Guasca

$d_3 = $ Cogua

#### 3. Conjunto de Estaciones de Servicio

$E$: Conjunto de estaciones de recarga de combustibles disponibles en la red.

Ejemplo: $E = {e_1, e_2, e_3}$ donde:

$e_1 = $ Estación La Estrella

$e_2 = $ Estación El Sol

$e_3 = $ Estación Caribe

#### 4. Conjunto de Vehículos

$V$: Conjunto de vehículos disponibles para el transporte.

Ejemplo: $V = {v_1, v_2, v_3}$ donde:

$v_1 = $ Camión de 30 ton

$v_2 = $ Camión de 25 ton

$v_3 = $ Camión de 20 ton

#### 5. Conjunto de "Arcos" Posibles

$A$: Conjunto de arcos entre nodos (puertos, estaciones, destinos) que representan posibles trayectos.

- Cada arco $(i, j)$ indica que es posible moverse del nodo $i$ al nodo $j$.

- Este conjunto se infiere del grafo de la red vial nacional y será determinado con base en la conectividad real y los peajes existentes.

#### 6. Conjunto de Municipios con Restricción de Peso

$M ⊆ D$: Subconjunto de destinos con restricción de peso para los vehículos.

Ejemplo: $E = {d_2, d_3}$ donde Guasca y Cogua tienen límites de peso.

### Definición de Parámetros
Lista de todas las constantes necesarias. Para este escenario, vamos a clasificar los parámetros en grupos de acuerdo con su tema, así serán más faciles de ubicar y diferenciar.

#### 1. Parámetros Geográficos

- $lat_i$: Latitud del nodo i
- $long_i$: Longitud del nodo i

#### 2. Red y Costos de Trayecto

- $dist_{ij}$: Distancia en km entre el nodo $i$ y el nodo $j$, con $(i, j) ∈ A$

- $PeajeBase_{ij}$: Tarifa base de peaje entre nodos $i$ y $j$ (COP).

- $PeajeTon_{ij}$: Costo adicional por tonelada transportada (COP/ton) entre $i$ y $j$
    - Si no aplica, se considera 0.

- $F_t$: Tarifa de flete por km (COP/km) -> 5.000

- $C_m$: Costo de mantenimiento por km (COP/km) -> 700

#### 3. Estaciones de servicio

Para cada estación $e ∈ E$:
- $PrecioCombustible_e$: Precio de combustible en estación $e$ (COP/galón)

#### 4. Vehículos

Para cada vehículo $v ∈ V$:
- $Capacidad_v$: Capacidad útil (ton)
- $Autonomía_v$: Autonomía mpaxima (km)

#### 5. Restricciones de Municipios

Para cada destino $d ∈ D$:
- $PesoMax_d$: Peso máximo permitido en el municipio $d$ (ton)
    - Si no tiene restricción, puede tomarse como un valor muy alto o inf.

#### 6. Parámetros Derivados o Preprocesados
Dependiendo del nivel de detalle del modelo, podríamos pensar en definir parámetros calculados como:
- $CostoTotal_{ijv} = dist_{ij} * (F_t + C_m) + PeajeBase_{ij} + PeajeTon_{ij} * Capacidad_v$

### Variables de Decisión
Detalle de las variables principales y las auxiliares que permiten capturar el comportamiento esperado del sistema. Para facilitar el reconocimiento de variables, las agruparemos en cuatro grupos clave.

#### 1. Variables de Ruta
Variables que nos indican si se utiliza un arco (trayecto) en una ruta para un vehículo determinado.

$$
x_{ij}^v ∈ \text{{0,1}}
$$

$$
  x_{ij}^v =
  \begin{cases}
  1 & \text{si el vehículo } v ∈ V \text{se desplaza del nodo $i$ al nodo $j$} \\
  0 & \text{dlc.}
  \end{cases}
$$

$$
  ∀ (i,j) ∈ A, v ∈ V
$$

#### 2. Variables de Carga Transportada
Variables que permiten modelar el peso que transporta cada vehículo por cada arco:

$$
  w_{ij}^v ≥ 0
$$

- Toneladas de carga transportadas por el vehículo $v$ en el arco $(i,j)$.
- Esto ayuda a validar restricciones de capacidad, peso en municipios, peajes variables por tonelada.

#### 3. Variables de Recarga
Variables que permiten representar la "estrategia" de recarga de combustible.

$$
  r_{i}^v ≥ 0
$$

- Galones de combustible recargados por el vehículo $v$ en la estación $i ∈ E$.
- Si el nodo no es una estación, entonces $r_{i}^v = 0$

#### 4. Variables Auxiliares (para consumo y autonomía)
Control del combustible disponible.

$$
  r_{i}^v ≥ 0
$$

- Cantidad de combustible (galones) disponibles en el tanque del vehículo $v$ al llegar al nodo $i$.

### Restricciones
Restricciones necesarias para representar adecuadamente la realidad del problema. Se agruparán las restricciones para mayor calidad.

#### 1. Restricciones de Flujo
Nos aseguramos de que cada vehículo tenga una ruta continua:

- **Conservación en nodos intermedios (flujo balanceado)**:

   $$
   \sum_{j:(j,i) ∈ A} x_{ij}^v = \sum_{j:(i,j) ∈ A} x_{ij}^v \quad \forall i \in N, v \in V
   $$
- **Salida del punto de acceso**:

   $$
   \sum_{j:(p,j) ∈ A} x_{pj}^v ≤ 1 \quad \forall p \in P, v \in V
   $$
- **Llegada a un destino**:

   $$
   \sum_{j:(i,d) ∈ A} x_{id}^v ≤ 1 \quad \forall d \in D, v \in V
   $$

#### 2. Restricciones de Capacidad y Peso
- **Límite de carga por vehículo**:

   $$
   \sum_{(i,j) ∈ A} w_{ij}^v ≤ Capacidad_v \quad \forall v \in V
   $$
- **Restricción de peso en municipios**:

   $$
   \sum_{i:(i,d) ∈ A} w_{id}^v ≤ PesoMax_d \quad \forall d \in (M ⊆ D), v \in V
   $$

#### 3. Restricciones de Autonomía y Consumo
- **Estado de combustible al llegar al nodo j**:

   $$
   f_{j}^v = f_{i}^v - Consumo_v * dist_{ij} + f_{j}^v \quad \forall (i,j) \in A
   $$
- **Autonomía máxima del taque**:

   $$
   f_{i}^v = Autonomía_v \quad \forall i \in N, v \in V
   $$
- **No recarga si el nodo no estación**:

   $$
   r_{i}^v = 0 \quad \forall i \notin E
   $$
#### 4. Cosotos de Peaje con peso
OJO: Esto se puede incorporar directamente en la función objetivo como término de costo
- **Si hay costo por tonelada**:
   $$
   Peaje_{ij}^v = PeajeBase_{ij} + PeajeTon_{ij} * w_{ij}^v
   $$
#### 5. Entrega Total de Carga
(Si se conoce la demanda total a transportar. Toca preguntar pq no entendí esa parte o no leí bien, la dejo por si acaso).

   $$
   \sum_{v ∈ V} \sum_{i:(i,d) ∈ A} w_{id}^v = Demanda_d \quad \forall d \in D
   $$
#### 6. Consistencia entre variables

- **Solo puede haber carga o recarga si el arco (ruta) es usado**:

   $$
   w_{ij}^v ≤ Capacidad_{v} * x_{ij}^v , r_{i}^v ≤ R_{max} * \sum_{j} x_{ij}^v
   $$
### Función objetivo
Planteamiento explícito de la función a optimizar considerando posibles penalizaciones por incumplimiento de restricciones.

$$∑\limits_{v∈V}∑\limits_{i∈N} ∑\limits_{j∈N} x_{ij}^v*CostoTotal_{ijv}$$

In [8]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_idaes()
    helper.install_ipopt()

!pip install -q pyomo
!apt-get install -y -qq glpk-utils
!pip install amplpy
!pip install highspy

--2025-05-27 23:01:25--  https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7171 (7.0K) [text/plain]
Saving to: ‘helper.py.2’

helper.py.2         100%[===================>]   7.00K  --.-KB/s    in 0s      

2025-05-27 23:01:25 (64.5 MB/s) - ‘helper.py.2’ saved [7171/7171]

IDAES found! No need to install.
Ipopt found! No need to install.
ipopt was successfully installed
k_aug was successfuly installed
cbc was successfuly installed
clp was successfuly installed
bonmin was successfuly installed
couenne was successfuly installed
ipopt_l1 was successfuly installed
 


In [9]:
import pandas as pd
from pyomo.environ import *
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import requests
import json
import folium
from pyomo.opt import SolverFactory
from amplpy import modules
import time
import random

##Data Preprocessing

In [10]:
api_key = "5b3ce3597851110001cf624835844b70fac7431981cedae2e64728c5"

###Preparing data for model

###Leer datos a df


In [11]:
vehicles_df = pd.read_csv('vehicles.csv')
clients_df = pd.read_csv('clients.csv')
depots_df = pd.read_csv('depots.csv')
stations_df = pd.read_csv('stations.csv')
#stations_df = stations_df.iloc[1:]
tolls_df = pd.read_csv('tolls.csv')


###Sacar todas las coordenadas

In [12]:
# Initialize geolocator
geolocator = Nominatim(user_agent="NationalLogistics")

# Function to get coordinates for a city
def get_coordinates(city):
    location = geolocator.geocode(city)
    if location:
        return [location.longitude, location.latitude]
    return None

# Get coordinates for all locations
coordinates = []
place_names = []
location_ids = [0]

# Add depot coordinates
depot_coords = [depots_df['Longitude'][0], depots_df['Latitude'][0]]
coordinates.append(depot_coords)
place_names.append('Depot')

# Add client coordinates
for _, row in clients_df.iterrows():
    city = row['City/Municipality']
    coords = get_coordinates(city)
    time.sleep(1)
    if coords:
        coordinates.append(coords)
        place_names.append(city)
        location_ids.append(location_ids[-1]+1)

# Add station coordinates
for _, row in stations_df.iterrows():
    coords = [row['Longitude'], row['Latitude']]
    coordinates.append(coords)
    place_names.append(f"Station {row['EstationID']}")
    location_ids.append(location_ids[-1]+1)
print(coordinates)
print(location_ids)

[[np.float64(-74.796387), np.float64(10.963889)], [-74.0836547, 4.6534109], [-75.6025597, 6.2697324], [-76.5325259, 3.4519988], [-75.5441671, 10.4265566], [-72.4689002, 8.0776187], [-73.1047294, 7.1669842], [-75.788322, 4.7854606], [-74.1950916, 11.2320944], [-75.2108857, 4.4386033], [-75.5081167, 5.0743694], [-75.2794303, 3.0332554], [-74.8231794, 11.0101922], [-73.4967836, 4.1114594], [-75.7413509, 4.4919761], [np.float64(-74.796387), np.float64(10.963889)], [np.float64(-75.51444), np.float64(10.39972)], [np.float64(-74.19904), np.float64(11.24079)], [np.float64(-73.1198), np.float64(7.12539)], [np.float64(-74.063644), np.float64(4.624335)], [np.float64(-75.590553), np.float64(6.230833)], [np.float64(-76.522224), np.float64(3.420556)], [np.float64(-74.4300155), np.float64(7.794112)], [np.float64(-75.19347), np.float64(8.597361)], [np.float64(-76.0563885), np.float64(4.8256945)], [np.float64(-75.292934), np.float64(4.0224455)], [np.float64(-73.2), np.float64(5.8)]]
[0, 1, 2, 3, 4, 5, 

###Calculamos matriz de distancias

In [13]:
# Function to get distance matrix from OpenRouteService
def get_distance_matrix(coords, api_key):
    url = "https://api.openrouteservice.org/v2/matrix/driving-car"
    payload = {
        "locations": coords,
        "metrics": ["distance"],
        "units": "km"
    }
    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Content-Type': 'application/json; charset=utf-8',
        'Authorization': api_key
    }

    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        matrix_data = response.json()
        return matrix_data.get('distances', [])
    except requests.exceptions.RequestException as e:
        print(f"Error making request to OpenRouteService: {e}")
        return None

# Get distance matrix
distance_matrix = get_distance_matrix(coordinates, api_key)


In [14]:
df = pd.DataFrame(distance_matrix, index=place_names, columns=place_names)
#df.drop('Station 1.0', inplace=True) #Quitar estacion en el puerto
#df.drop(columns='Station 1.0', inplace=True) #Quitar
df

,Depot,Bogotá,Medellín,Cali,Cartagena,Cúcuta,Bucaramanga,Pereira,Santa Marta,Ibagué,...,Station 3.0,Station 4.0,Station 5.0,Station 6.0,Station 7.0,Station 8.0,Station 9.0,Station 10.0,Station 11.0,Station 12.0
Depot,0.00,990.65,701.20,1131.66,134.70,671.58,581.73,954.35,99.16,991.47,...,100.89,649.32,996.00,701.08,1136.08,759.31,356.02,967.64,1098.41,856.99
Bogotá,985.65,0.00,411.26,452.24,1035.59,586.12,432.51,317.56,943.25,190.93,...,944.97,413.75,7.29,407.77,456.66,625.73,752.91,350.62,222.22,184.59
Medellín,700.92,402.34,0.00,438.77,659.35,613.67,404.18,261.46,828.87,375.31,...,830.60,389.88,407.69,6.91,443.19,511.35,420.71,274.75,510.09,413.80
Cali,1140.67,451.18,445.09,0.00,1099.11,998.63,789.14,208.22,1213.83,260.84,...,1215.56,774.84,453.46,440.57,4.75,896.31,860.46,213.10,378.16,646.09
Cartagena,131.32,1039.18,659.28,1089.74,0.00,720.11,630.26,912.43,231.19,1040.01,...,232.91,697.86,1044.54,659.17,1094.17,807.84,314.11,925.73,1146.94,905.52
Cúcuta,670.59,586.00,613.06,1000.35,720.53,0.00,242.47,866.20,628.18,732.23,...,629.91,223.69,590.18,612.94,1004.77,532.38,549.67,879.50,839.16,416.09
Bucaramanga,581.19,426.40,404.28,791.57,631.14,238.68,0.00,657.43,538.79,523.45,...,540.52,14.34,430.58,404.16,795.99,323.60,460.27,670.72,630.38,257.64
Pereira,961.19,309.96,265.61,206.09,919.62,857.41,647.92,0.00,1072.61,119.62,...,1074.34,633.62,312.24,261.08,210.51,755.09,680.98,54.32,236.94,504.87
Santa Marta,99.33,949.79,831.45,1218.74,234.26,630.72,540.87,1084.59,0.00,950.62,...,1.76,608.46,955.14,831.33,1223.16,718.45,448.48,1097.89,1057.55,816.13
Ibagué,991.15,193.18,405.58,263.43,1041.09,733.54,524.05,128.76,948.74,0.00,...,950.47,509.76,195.46,402.09,267.85,631.23,747.23,161.81,120.16,388.09


###Preparamos conjuntos y parametros pyomo

In [15]:
#1 Conjunto de Puntos de Acceso (Puertos)
puertos = [0]
#2 Conjunto de Destinos (Centros de Consumo)
clientes = clients_df['ClientID'].values.tolist()
#3 Conjunto de Estaciones de Servicio
estaciones = location_ids[clientes[-1]+1:]
#4 Conjunto de Vehículos
vehiculos = vehicles_df['VehicleID'].values.tolist()
#5 Conjunto de "Arcos" Posibles
nodos = location_ids
arcos = [(i,j) for i in nodos for j in nodos if i!=j]
#6 Conjunto de Municipios con Restricción de Peso
clientes_con_restriccion = clients_df[clients_df['MaxWeight'].notna()]['LocationID'].values.tolist()
print(puertos)
print(clientes)
print(estaciones)
print(vehiculos)
print(nodos)
#print(arcos)
print(clientes_con_restriccion)

[0]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
[1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
[4, 6, 8, 9, 10, 11, 12, 13, 14]


In [16]:
#1. Parametros geograficos
print(len(distance_matrix))
df = pd.DataFrame(distance_matrix, index=location_ids, columns=location_ids)
df

27


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,0.00,990.65,701.20,1131.66,134.70,671.58,581.73,954.35,99.16,991.47,...,100.89,649.32,996.00,701.08,1136.08,759.31,356.02,967.64,1098.41,856.99
1,985.65,0.00,411.26,452.24,1035.59,586.12,432.51,317.56,943.25,190.93,...,944.97,413.75,7.29,407.77,456.66,625.73,752.91,350.62,222.22,184.59
2,700.92,402.34,0.00,438.77,659.35,613.67,404.18,261.46,828.87,375.31,...,830.60,389.88,407.69,6.91,443.19,511.35,420.71,274.75,510.09,413.80
3,1140.67,451.18,445.09,0.00,1099.11,998.63,789.14,208.22,1213.83,260.84,...,1215.56,774.84,453.46,440.57,4.75,896.31,860.46,213.10,378.16,646.09
4,131.32,1039.18,659.28,1089.74,0.00,720.11,630.26,912.43,231.19,1040.01,...,232.91,697.86,1044.54,659.17,1094.17,807.84,314.11,925.73,1146.94,905.52
5,670.59,586.00,613.06,1000.35,720.53,0.00,242.47,866.20,628.18,732.23,...,629.91,223.69,590.18,612.94,1004.77,532.38,549.67,879.50,839.16,416.09
6,581.19,426.40,404.28,791.57,631.14,238.68,0.00,657.43,538.79,523.45,...,540.52,14.34,430.58,404.16,795.99,323.60,460.27,670.72,630.38,257.64
7,961.19,309.96,265.61,206.09,919.62,857.41,647.92,0.00,1072.61,119.62,...,1074.34,633.62,312.24,261.08,210.51,755.09,680.98,54.32,236.94,504.87
8,99.33,949.79,831.45,1218.74,234.26,630.72,540.87,1084.59,0.00,950.62,...,1.76,608.46,955.14,831.33,1223.16,718.45,448.48,1097.89,1057.55,816.13
9,991.15,193.18,405.58,263.43,1041.09,733.54,524.05,128.76,948.74,0.00,...,950.47,509.76,195.46,402.09,267.85,631.23,747.23,161.81,120.16,388.09


In [17]:
#2. Red y Costos de Trayecto
peajes_base_dict = {}
peajes_ton_dict = {}
for cliente in clientes:
    peajes_base_dict[cliente] = tolls_df['BaseRate'].values[0]
    peajes_ton_dict[cliente] = tolls_df['RatePerTon'].values[0]
print(peajes_base_dict)
print(peajes_ton_dict)

flete_por_km = 5000
mantenimiento_por_km = 700
consumo_combustible = 0.2  # (valores asumidos)


{1: np.float64(25000.0), 2: np.float64(25000.0), 3: np.float64(25000.0), 4: np.float64(25000.0), 5: np.float64(25000.0), 6: np.float64(25000.0), 7: np.float64(25000.0), 8: np.float64(25000.0), 9: np.float64(25000.0), 10: np.float64(25000.0), 11: np.float64(25000.0), 12: np.float64(25000.0), 13: np.float64(25000.0), 14: np.float64(25000.0)}
{1: np.float64(800.0), 2: np.float64(800.0), 3: np.float64(800.0), 4: np.float64(800.0), 5: np.float64(800.0), 6: np.float64(800.0), 7: np.float64(800.0), 8: np.float64(800.0), 9: np.float64(800.0), 10: np.float64(800.0), 11: np.float64(800.0), 12: np.float64(800.0), 13: np.float64(800.0), 14: np.float64(800.0)}


In [18]:
#3. Estaciones de servicio
estaciones_precio_dict = stations_df['FuelCost'].to_list()
estaciones_precio_dict = dict(zip(estaciones, estaciones_precio_dict))
print(estaciones_precio_dict)

{15: 13500.0, 16: 14000.0, 17: 13800.0, 18: 15200.0, 19: 14800.0, 20: 15500.0, 21: 16000.0, 22: 16500.0, 23: 16200.0, 24: 15800.0, 25: 15300.0, 26: 14900.0}


In [19]:
#4. Vehículos
capacidad_dict = vehicles_df.set_index('VehicleID')['Capacity'].to_dict()
print(capacidad_dict)
rango_dict = vehicles_df.set_index('VehicleID')['Range'].to_dict()
print(rango_dict)


{1: 80.0, 2: 60.0, 3: 50.0, 4: 40.0, 5: 30.0, 6: 10.0}
{1: 1720, 2: 1510, 3: 1300, 4: 1500, 5: 870, 6: 1200}


In [20]:
#5. Restricciones de Municipios
demanda_dict = clients_df.set_index('ClientID')['Demand'].to_dict()
print(demanda_dict)
peso_max_df = clients_df.set_index('ClientID')['MaxWeight']
peso_max_df.dropna(inplace=True)
peso_max_dict = peso_max_df.to_dict()
print(peso_max_dict)



{1: 16.0, 2: 18.0, 3: 16.0, 4: 18.0, 5: 15.0, 6: 17.6, 7: 17.6, 8: 10.0, 9: 11.0, 10: 9.0, 11: 5.0, 12: 7.0, 13: 5.0, 14: 10.0}
{3: 35.0, 5: 35.0, 7: 25.0, 8: 50.0, 9: 40.0, 10: 38.0, 11: 10.0, 12: 50.0, 13: 10.0}


In [21]:
# Resumen
print("puertos:", puertos)
print("clientes:", clientes)
print("estaciones:", estaciones)
print("vehiculos:", vehiculos)
print("nodos:", nodos)
print("clientes_con_restriccion:", clientes_con_restriccion)
print("distance_matrix:", distance_matrix)
print("peajes_base_dict:", peajes_base_dict)
print("peajes_ton_dict:", peajes_ton_dict)
print("estaciones_precio_dict:", estaciones_precio_dict)
print("capacidad_dict:", capacidad_dict)
print("rango_dict:", rango_dict)
print("demanda_dict:", demanda_dict)
print("peso_max_dict:", peso_max_dict)


puertos: [0]
clientes: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
estaciones: [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
vehiculos: [1, 2, 3, 4, 5, 6]
nodos: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
clientes_con_restriccion: [4, 6, 8, 9, 10, 11, 12, 13, 14]
distance_matrix: [[0.0, 990.65, 701.2, 1131.66, 134.7, 671.58, 581.73, 954.35, 99.16, 991.47, 913.99, 1174.41, 8.07, 1113.0, 1088.47, 0.0, 131.44, 100.89, 649.32, 996.0, 701.08, 1136.08, 759.31, 356.02, 967.64, 1098.41, 856.99], [985.65, 0.0, 411.26, 452.24, 1035.59, 586.12, 432.51, 317.56, 943.25, 190.93, 294.54, 298.22, 992.9, 132.65, 281.11, 985.65, 1031.93, 944.97, 413.75, 7.29, 407.77, 456.66, 625.73, 752.91, 350.62, 222.22, 184.59], [700.92, 402.34, 0.0, 438.77, 659.35, 613.67, 404.18, 261.46, 828.87, 375.31, 221.1, 586.09, 721.02, 524.69, 318.57, 700.92, 655.69, 830.6, 389.88, 407.69, 6.91, 443.19, 511.35, 420.71, 274.75, 510.09, 413.8], [1140.67, 451.18, 44

In [22]:

# Check if total demand is less than total capacity
total_demand = sum(demanda_dict.values())
total_capacity = sum(capacidad_dict.values())
print(f"Total demand: {total_demand}")
print(f"Total capacity: {total_capacity}")

if total_demand > total_capacity:
    print("Error: Total demand exceeds total vehicle capacity.")

# Check for any max weight less than demand
for client_id, demand in demanda_dict.items():
    if client_id in peso_max_dict and peso_max_dict[client_id] < demand:
        print(f"Error: Max weight for client {client_id} is less than its demand.")


Total demand: 175.2
Total capacity: 270.0


#Modelo


##Caso 2

In [ ]:
# MODELO PYOMO

indices_estaciones = estaciones
n = len(nodos)
autonomia = rango_dict
cost_matrix = distance_matrix
demanda = demanda_dict
capacidad = capacidad_dict
peso_max = peso_max_dict
precio_combustible = estaciones_precio_dict
consumo_km = consumo_combustible

model = ConcreteModel()
model.N = RangeSet(0, n - 1) # todos los nodos
model.C = RangeSet(1, 14) # Clientes
model.K = RangeSet(1, len(vehiculos)) # Vehiculos


model.x = Var(model.N, model.N, model.K, within=Binary)
model.u = Var(model.N, model.K, within=NonNegativeReals)

model.fuel = Var(model.N, model.K, domain=NonNegativeReals)
model.refuel = Var(model.N, model.K, domain=NonNegativeReals)

for k in model.K:
    model.fuel[0, k].fix(autonomia[k])

# Objetivo: minimizar distancia y costo de gasolina

model.obj = Objective(
    expr=sum(
        cost_matrix[i][j] * model.x[i, j, k]
        for i in model.N for j in model.N for k in model.K if i != j
    ) +
    sum(
        precio_combustible.get(i, 0) * model.refuel[i, k]
        for i in indices_estaciones for k in model.K
    ),
    sense=minimize
)


# Restricciones

# Se asegura que un camión salga una vez del depósito y regrese una vez
model.deposito = ConstraintList()
for k in model.K:
    model.deposito.add(sum(model.x[0, j, k] for j in model.N if j != 0) == 1)
    model.deposito.add(sum(model.x[j, 0, k] for j in model.N if j != 0) == 1)

#Asegura conservación de flujo: si un camión entra a un nodo, también debe salir. ( no teletransporte)
model.flujo = ConstraintList()
for k in model.K:
    for i in model.N:
        if i != 0:
            model.flujo.add(
                sum(model.x[i, j, k] for j in model.N if i != j) ==
                sum(model.x[j, i, k] for j in model.N if i != j)
            )

# Evita los ciclos
model.subtour = ConstraintList()
for k in model.K:
    for i in model.N:
        for j in model.N:
            if i != j and i != 0 and j != 0:
                model.subtour.add(
                    model.u[i, k] - model.u[j, k] + n * model.x[i, j, k] <= n - 1
                )

# Cada cliente debe ser visitado exactamento una vez
model.visita = ConstraintList()
for i in model.C:
    if i != 0:
        model.visita.add(
            sum(model.x[i, j, k] for j in model.N for k in model.K if i != j) == 1
       )

#Asegura que la demanda total que lleva cada camión no supere su capacidad máxima de carga
model.capacidad_total = ConstraintList()
for k in model.K:
    model.capacidad_total.add(
        sum(
            demanda.get(i, 0) * sum(model.x[i, j, k] for j in model.N if i != j)
            for i in model.C
        ) <= capacidad[k]
    )

# Restricciones Gasolina

# Simula la dinámica del combustible: debo tener suficiente para ir de tal a tal
model.restriccion_combustible = ConstraintList()
for k in model.K:
    for i in model.N:
        for j in model.N:
            if i != j:
                distancia = cost_matrix[i][j]
                model.restriccion_combustible.add(
                    model.fuel[j, k] <= model.fuel[i, k] - consumo_km * distancia * model.x[i, j, k] + model.refuel[j, k]
                )

# Combustible nunca puede superar el tanque
model.limite_tanque = ConstraintList()
for k in model.K:
    for i in model.N:
        model.limite_tanque.add(model.fuel[i, k] <= autonomia[k])

#Cambios para caso 3



# SOLVER

solver = SolverFactory('highs')

results = solver.solve(model, tee=True, options={"time_limit": 10*60})

# IMPRIMIR RUTAS

routes = {k: [] for k in model.K}
for k in model.K:
    actual = 0
    visitados = set()
    while True:
        next_node = None
        for j in model.N:
            if j != actual and model.x[actual, j, k].value is not None and model.x[actual, j, k].value > 0.5:
                next_node = j
                break
        if next_node is None or next_node in visitados:
            break
        routes[k].append(next_node)
        visitados.add(next_node)
        actual = next_node

for k, ruta in routes.items():
    traducida = [i for i in [0] + ruta]
    print(f"Camión {k}: {' → '.join(map(str, traducida))}")



###Caso 3

In [43]:
# MODELO PYOMO

indices_estaciones = estaciones
n = len(nodos)
autonomia = rango_dict
cost_matrix = distance_matrix
demanda = demanda_dict
capacidad = capacidad_dict
peso_max = peso_max_dict
precio_combustible = estaciones_precio_dict
consumo_km = consumo_combustible

model = ConcreteModel()
model.N = RangeSet(0, n - 1) # todos los nodos
model.C = RangeSet(1, 14) # Clientes
model.K = RangeSet(1, len(vehiculos)) # Vehiculos

model.x = Var(model.N, model.N, model.K, within=Binary)
model.u = Var(model.N, model.K, within=NonNegativeReals)

model.fuel = Var(model.N, model.K, domain=NonNegativeReals)
model.refuel = Var(model.N, model.K, domain=NonNegativeReals)

# Variable auxiliar para la carga transportada por cada camión
model.carga_camion = Var(model.K, domain=NonNegativeReals)

# Variable auxiliar para linearizar peajes: carga_camion[k] * x[i,j,k]
M = 10000  # Big M constant
model.peaje_aux = Var(model.N, model.N, model.K, domain=NonNegativeReals)

for k in model.K:
    model.fuel[0, k].fix(autonomia[k])

# Calcular la carga total por camión
model.carga_total = ConstraintList()
for k in model.K:
    model.carga_total.add(
        model.carga_camion[k] == sum(
            demanda.get(i, 0) * sum(model.x[i, j, k] for j in model.N if i != j)
            for i in model.C
        )
    )

# Restricciones para linearizar peajes: peaje_aux[i,j,k] = carga_camion[k] * x[i,j,k]
model.linearizacion_peajes = ConstraintList()
for k in model.K:
    for i in model.N:
        for j in model.N:
            if i != j and j in peajes_base_dict:
                # peaje_aux[i,j,k] <= M * x[i,j,k]
                model.linearizacion_peajes.add(model.peaje_aux[i, j, k] <= M * model.x[i, j, k])
                # peaje_aux[i,j,k] <= carga_camion[k]
                model.linearizacion_peajes.add(model.peaje_aux[i, j, k] <= model.carga_camion[k])
                # peaje_aux[i,j,k] >= carga_camion[k] - M * (1 - x[i,j,k])
                model.linearizacion_peajes.add(model.peaje_aux[i, j, k] >= model.carga_camion[k] - M * (1 - model.x[i, j, k]))
                # peaje_aux[i,j,k] >= 0 (ya está implícito en el domain)

# Objetivo linearizado: minimizar distancia, costo de gasolina y peajes
model.obj = Objective(
    expr=sum(
        cost_matrix[i][j] * model.x[i, j, k]
        for i in model.N for j in model.N for k in model.K if i != j
    ) +
    sum(
        precio_combustible.get(i, 0) * model.refuel[i, k]
        for i in indices_estaciones for k in model.K
    ) +
    sum(
        peajes_base_dict.get(j, 0) * model.x[i, j, k] + peajes_ton_dict.get(j, 0) * model.peaje_aux[i, j, k]
        for i in model.N for j in model.N for k in model.K
        if i != j and j in peajes_base_dict
    ),
    sense=minimize
)

# Restricciones

# Se asegura que un camión salga una vez del depósito y regrese una vez
model.deposito = ConstraintList()
for k in model.K:
    model.deposito.add(sum(model.x[0, j, k] for j in model.N if j != 0) == 1)
    model.deposito.add(sum(model.x[j, 0, k] for j in model.N if j != 0) == 1)

#Asegura conservación de flujo: si un camión entra a un nodo, también debe salir. ( no teletransporte)
model.flujo = ConstraintList()
for k in model.K:
    for i in model.N:
        if i != 0:
            model.flujo.add(
                sum(model.x[i, j, k] for j in model.N if i != j) ==
                sum(model.x[j, i, k] for j in model.N if i != j)
            )

# Evita los ciclos
model.subtour = ConstraintList()
for k in model.K:
    for i in model.N:
        for j in model.N:
            if i != j and i != 0 and j != 0:
                model.subtour.add(
                    model.u[i, k] - model.u[j, k] + n * model.x[i, j, k] <= n - 1
                )

# Cada cliente debe ser visitado exactamento una vez
model.visita = ConstraintList()
for i in model.C:
    if i != 0:
        model.visita.add(
            sum(model.x[i, j, k] for j in model.N for k in model.K if i != j) == 1
       )

#Asegura que la demanda total que lleva cada camión no supere su capacidad máxima de carga
model.capacidad_total = ConstraintList()
for k in model.K:
    model.capacidad_total.add(model.carga_camion[k] <= capacidad[k])

# Restricciones Gasolina
model.restriccion_combustible = ConstraintList()
for k in model.K:
    for i in model.N:
        for j in model.N:
            if i != j:
                distancia = cost_matrix[i][j]
                model.restriccion_combustible.add(
                    model.fuel[j, k] <= model.fuel[i, k] - consumo_km * distancia * model.x[i, j, k] + model.refuel[j, k]
                )

# Combustible nunca puede superar el tanque
model.limite_tanque = ConstraintList()
for k in model.K:
    for i in model.N:
        model.limite_tanque.add(model.fuel[i, k] <= autonomia[k])

#Cambios para caso 3

# Restricciones de peso por municipio
model.peso_municipio = ConstraintList()
for k in model.K:
    for i in clientes_con_restriccion:
        # Solo aplicar restricción si el vehículo visita el municipio
        model.peso_municipio.add(
            model.carga_camion[k] <= peso_max.get(i, float('inf')) + M * (1 - sum(model.x[l, i, k] for l in model.N if l != i))
        )

# SOLVER
solver = SolverFactory('highs')
results = solver.solve(model, tee=True, options={"time_limit": 10*60})

print(f"Solver status: {results.solver.status}")
print(f"Termination condition: {results.solver.termination_condition}")

# IMPRIMIR RUTAS
routes = {k: [] for k in model.K}
for k in model.K:
    actual = 0
    visitados = set()
    while True:
        next_node = None
        for j in model.N:
            if j != actual and model.x[actual, j, k].value is not None and model.x[actual, j, k].value > 0.5:
                next_node = j
                break
        if next_node is None or next_node in visitados:
            break
        routes[k].append(next_node)
        visitados.add(next_node)
        actual = next_node

for k, ruta in routes.items():
    traducida = [i for i in [0] + ruta]
    print(f"Camión {k}: {' → '.join(map(str, traducida))}")

Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 15074 rows; 6882 cols; 57930 nonzeros; 4212 integer variables (4212 binary)
Coefficient ranges:
  Matrix [3e-01, 1e+04]
  Cost   [2e+00, 3e+04]
  Bound  [1e+00, 2e+03]
  RHS    [1e+00, 1e+04]
Presolving model
12548 rows, 6786 cols, 48168 nonzeros  0s
10874 rows, 5546 cols, 40334 nonzeros  4s
10570 rows, 5546 cols, 40030 nonzeros  5s

Solving MIP model with:
   10570 rows
   5546 cols (3582 binary, 0 integer, 86 implied int., 1878 continuous)
   40030 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQ

##Resultados

In [44]:
def verificar_resultados(model):
    print("\n=== VERIFICACIÓN BÁSICA DEL MODELO ===")

    # 1. Estado de la solución (forma correcta)
    if (results.solver.status == SolverStatus.ok and
        results.solver.termination_condition == TerminationCondition.optimal):
        print("✓ Solución óptima encontrada")
    elif results.solver.termination_condition == TerminationCondition.infeasible:
        print("✗ El problema es infactible - no existe solución que cumpla todas las restricciones")
        return
    else:
        print("⚠ Solución encontrada pero no necesariamente óptima")
        print(f"Estado del solver: {results.solver.status}")
        print(f"Condición de terminación: {results.solver.termination_condition}")

verificar_resultados(model)


=== VERIFICACIÓN BÁSICA DEL MODELO ===
⚠ Solución encontrada pero no necesariamente óptima
Estado del solver: aborted
Condición de terminación: maxTimeLimit


In [45]:
# 4. Trajectory of each vehicle
print("\nTrayectoria de cada vehículo:")
arcos_por_vehiculo = {v: [] for v in model.K}

for i in model.N:
  for j in model.N:
    if i ==j:
      continue
    for v in model.K:
        if model.x[i, j, v].value and model.x[i, j, v].value > 0.5:
            #print(f"Vehículo {v} viaja de {i} a {j}")
            arcos_por_vehiculo[v].append((i, j))
for v, arcs in arcos_por_vehiculo.items():
    print(f"Vehículo {v}: {arcs}")


Trayectoria de cada vehículo:
Vehículo 1: [(0, 12), (3, 21), (10, 3), (12, 24), (21, 0), (24, 10)]
Vehículo 2: [(0, 18), (1, 6), (6, 15), (15, 0), (18, 1)]
Vehículo 3: [(0, 23), (2, 25), (4, 0), (23, 2), (25, 4)]
Vehículo 4: [(0, 22), (7, 14), (8, 7), (14, 20), (20, 0), (22, 8)]
Vehículo 5: [(0, 26), (5, 0), (9, 18), (18, 5), (26, 9)]
Vehículo 6: [(0, 19), (11, 18), (13, 0), (18, 13), (19, 11)]


In [52]:
import folium
from pyomo.environ import value
import random

# Initialize the map at a central location
m = folium.Map(location=[4.60971, -74.08175], zoom_start=10)

# Convert coordinates to (latitude, longitude) format
coords_long_lat = [tuple(reversed(coord)) for coord in coordinates]
node_coords = dict(zip(nodos, coords_long_lat))

# Add depots to the map
for depot, coords in node_coords.items():
    if depot in puertos:
        folium.Marker(location=coords, popup=f"Depot: {depot}", icon=folium.Icon(color="white")).add_to(m)

# Add stations to the map
for station, coords in node_coords.items():
    if station in estaciones:
        folium.Marker(location=coords, popup=f"Station: Location ID{station}", icon=folium.Icon(color="blue")).add_to(m)

# Add destinations to the map
for client, coords in node_coords.items():
    if client in clientes:
        folium.Marker(
            location=coords,
            popup=f"{client}",
            icon=folium.Icon(color="green"),
        ).add_to(m)

# Add the trajectories to the map using arcos_por_vehiculo
for v, arcs in arcos_por_vehiculo.items():
    # Convert arcs to coordinates
    if not arcs:
      continue
    vehicle_routes = [(node_coords[i], node_coords[j]) for (i, j) in arcs if i in node_coords and j in node_coords]

    # Assign a random color to each vehicle's route
    color = "#" + ''.join(random.choices('0123456789ABCDEF', k=6))

    # Draw the trajectory as a PolyLine
    folium.PolyLine(
        [point for route in vehicle_routes for point in route],  # Flatten the list of routes
        color=color,
        weight=2.5,
        opacity=1,
        tooltip=f"Vehicle {v}",
    ).add_to(m)

# Save the map to an HTML file
m

In [75]:
import csv
import pandas as pd

def generate_results_csv(model, routes, filename="vehicle_routes_results.csv"):
    """
    Generate CSV file with detailed vehicle route results
    """
    results_data = []

    for k in model.K:
        if not routes[k]:  # Skip empty routes
            continue

        # Basic vehicle info
        vehicle_id = f"CAM{k:03d}"
        load_cap = capacidad[k] * 1000  # Convert to kg
        fuel_cap = autonomia[k]  # Already in appropriate units

        # Build route sequence
        route_sequence = build_route_sequence(routes[k])

        # Count municipalities visited
        municipalities_count = len([node for node in routes[k] if node in model.C])

        # Calculate demands satisfied at each point
        demand_satisfied = calculate_demand_sequence(model, routes[k], k)

        # Initial conditions
        init_load = sum(demanda.get(i, 0) for i in routes[k] if i in model.C) * 1000  # Convert to kg
        init_fuel = autonomia[k]

        # Refuel information - CALCULATE BASED ON DISTANCE TO NEXT NODE
        refuel_stops, refuel_amounts = calculate_refuel_info_distance_based(model, routes[k], k)

        # Toll information
        tolls_visited, toll_costs = calculate_toll_info(model, routes[k], k)

        # Vehicle weights at each municipality
        vehicle_weights = calculate_vehicle_weights(model, routes[k], k)

        # Distance calculation
        total_distance = calculate_total_distance(routes[k])

        # Time calculation (assuming average speed of 60 km/h)
        total_time = total_distance / 60 * 60  # Convert to minutes

        # Cost calculations
        fuel_cost = calculate_fuel_cost_distance_based(model, routes[k], k)
        toll_cost = calculate_toll_cost(model, routes[k], k)
        total_cost = fuel_cost + toll_cost + total_distance * 100  # Adding distance cost

        # Prepare row data
        row = {
            'VehicleId': vehicle_id,
            'LoadCap': int(load_cap),
            'FuelCap': int(fuel_cap),
            'RouteSeq': route_sequence,
            'Municipalities': municipalities_count,
            'Demand': demand_satisfied,
            'InitLoad': int(init_load),
            'InitFuel': int(init_fuel),
            'RefuelStops': refuel_stops,
            'RefuelAmounts': refuel_amounts,
            'TollsVisited': tolls_visited,
            'TollCosts': toll_costs,
            'VehicleWeights': vehicle_weights,
            'Distance': round(total_distance, 1),
            'Time': round(total_time, 1),
            'FuelCost': int(fuel_cost),
            'TollCost': int(toll_cost),
            'TotalCost': int(total_cost)
        }

        results_data.append(row)

    # Create DataFrame and save to CSV
    df = pd.DataFrame(results_data)
    df.to_csv(filename, index=False)

    return df

def build_route_sequence(route):
    """Build route sequence string with proper identifiers"""
    sequence = ["PTO"]

    for node in route:
        if node in range(1, 15):  # Clients/Municipalities
            sequence.append(f"MUN{node:02d}")
        elif node in range(15, 27):  # Gas stations
            sequence.append(f"EST{node-14:02d}")

    sequence.append("PTO")
    return " - ".join(sequence)

def calculate_demand_sequence(model, route, k):
    """Calculate demand satisfied at each point in route"""
    demands = []

    for node in route:
        if node in model.C:
            # Check if this vehicle visits this client
            visits_client = any(model.x[node, j, k].value > 0.5 for j in model.N if node != j)
            if visits_client:
                demands.append(str(int(demanda.get(node, 0) * 1000)))  # Convert to kg
            else:
                demands.append("0")
        else:
            demands.append("0")  # Gas stations and tolls have 0 demand

    return "-".join(demands)

def calculate_refuel_info_distance_based(model, route, k):
    """Calculate refuel stops and amounts based on distance to next node"""
    refuel_stops = 0
    refuel_amounts = []

    # Create full route including depot
    full_route = [0] + route + [0]
    current_fuel = autonomia[k]  # Start with full tank

    for i in range(len(full_route) - 1):
        current_node = full_route[i]
        next_node = full_route[i + 1]

        # Calculate fuel needed to reach next node
        distance_to_next = cost_matrix[current_node][next_node]
        fuel_needed = consumo_km * distance_to_next

        # If current node is a gas station and we need fuel
        if current_node in indices_estaciones:  # 20% safety margin
            #print("Refuel Stop")
            refuel_stops += 1
            # Refuel enough to reach next node plus some reserve
            fuel_to_add = min(autonomia[k] - current_fuel, fuel_needed * 2)  # Don't exceed tank capacity
            refuel_amounts.append(str(int(fuel_to_add)))
            current_fuel += fuel_to_add

        # Consume fuel to reach next node
        current_fuel -= fuel_needed

        # Ensure we don't go negative
        current_fuel = max(0, current_fuel)

    refuel_amounts_str = "-".join(refuel_amounts) if refuel_amounts else "0"
    return refuel_stops, refuel_amounts_str

def calculate_toll_info(model, route, k):
    """Calculate toll information"""
    tolls_visited = 0
    toll_costs = []

    full_route = [0] + route + [0]

    for i in range(len(full_route) - 1):
        current_node = full_route[i]
        next_node = full_route[i + 1]

        if next_node in peajes_base_dict:
            if hasattr(model.x[current_node, next_node, k], 'value') and model.x[current_node, next_node, k].value > 0.5:
                tolls_visited += 1
                # Calculate toll cost for this specific passage
                base_cost = peajes_base_dict.get(next_node, 0)
                if hasattr(model.carga_camion[k], 'value') and model.carga_camion[k].value is not None:
                    weight_cost = peajes_ton_dict.get(next_node, 0) * model.carga_camion[k].value
                else:
                    weight_cost = 0
                total_toll_cost = base_cost + weight_cost
                toll_costs.append(str(int(total_toll_cost)))

    toll_costs_str = "-".join(toll_costs) if toll_costs else "0"
    return tolls_visited, toll_costs_str

def calculate_vehicle_weights(model, route, k):
    """Calculate vehicle weights at each municipality"""
    weights = []

    if hasattr(model.carga_camion[k], 'value') and model.carga_camion[k].value is not None:
        current_load = model.carga_camion[k].value * 1000  # Convert to kg
    else:
        current_load = 0

    for node in route:
        if node in model.C:
            weights.append(str(int(current_load)))
            # Subtract delivered demand
            current_load -= demanda.get(node, 0) * 1000

    return "-".join(weights) if weights else "0"

def calculate_total_distance(route):
    """Calculate total distance for the route"""
    if not route:
        return 0.0

    total_dist = 0.0
    full_route = [0] + route + [0]

    for i in range(len(full_route) - 1):
        current = full_route[i]
        next_node = full_route[i + 1]
        total_dist += cost_matrix[current][next_node]

    return total_dist

def calculate_fuel_cost_distance_based(model, route, k):
    """Calculate total fuel cost for vehicle k based on distance calculations"""
    total_cost = 0.0

    # Create full route including depot
    full_route = [0] + route + [0]
    current_fuel = autonomia[k]  # Start with full tank

    for i in range(len(full_route) - 1):
        current_node = full_route[i]
        next_node = full_route[i + 1]

        # Calculate fuel needed to reach next node
        distance_to_next = cost_matrix[current_node][next_node]
        fuel_needed = consumo_km * distance_to_next

        # If current node is a gas station and we need fuel
        if current_node in indices_estaciones:  # 20% safety margin
            # Get the price for this station
            station_index = indices_estaciones.index(current_node)
            station_id = estaciones[station_index]
            station_price = precio_combustible.get(station_id, 0)

            # Calculate refuel amount and cost
            fuel_to_add = min(autonomia[k] - current_fuel, fuel_needed * 2)
            cost = station_price * fuel_to_add
            total_cost += cost
            current_fuel += fuel_to_add

        # Consume fuel to reach next node
        current_fuel -= fuel_needed
        current_fuel = max(0, current_fuel)

    return total_cost

def calculate_toll_cost(model, route, k):
    """Calculate total toll cost for vehicle k"""
    total_cost = 0.0
    full_route = [0] + route + [0]

    for i in range(len(full_route) - 1):
        current_node = full_route[i]
        next_node = full_route[i + 1]

        if next_node in peajes_base_dict:
            if hasattr(model.x[current_node, next_node, k], 'value') and model.x[current_node, next_node, k].value > 0.5:
                base_cost = peajes_base_dict.get(next_node, 0)
                if hasattr(model.carga_camion[k], 'value') and model.carga_camion[k].value is not None:
                    weight_cost = peajes_ton_dict.get(next_node, 0) * model.carga_camion[k].value
                else:
                    weight_cost = 0
                total_cost += base_cost + weight_cost

    return total_cost

# Generate the CSV file
results_df = generate_results_csv(model, routes, "caso_c_results.csv")

In [76]:
results_df

,VehicleId,LoadCap,FuelCap,RouteSeq,Municipalities,Demand,InitLoad,InitFuel,RefuelStops,RefuelAmounts,TollsVisited,TollCosts,VehicleWeights,Distance,Time,FuelCost,TollCost,TotalCost
0,CAM001,80000,1720,PTO - MUN12 - EST10 - MUN10 - MUN03 - EST07 - PTO,3,7000-0-9000-16000-0-0,32000,1720,2,49-227,3,50600-50600-50600,32000-25000-16000,2529.4,2529.4,4421694,151800,4826434
1,CAM002,60000,1510,PTO - EST04 - MUN01 - MUN06 - EST01 - PTO,2,0-16000-17600-0-0,33600,1510,2,129-0,2,51880-51880,33600-17600,2075.3,2075.3,1973932,103760,2285221
2,CAM003,50000,1300,PTO - EST09 - MUN02 - EST11 - MUN04 - PTO,2,0-18000-0-18000-0,36000,1300,2,71-186,2,53800-53800,36000-18000,2559.2,2559.2,4003497,107600,4367013
3,CAM004,40000,1500,PTO - EST08 - MUN08 - MUN07 - MUN14 - EST06 - PTO,3,0-10000-17600-10000-0-0,37600,1500,2,151-281,3,55080-55080-55080,37600-27600-10000,3631.5,3631.5,6866927,165240,7395317
4,CAM005,30000,870,PTO - EST12 - MUN09 - EST04 - MUN05 - PTO,2,0-11000-0-15000-0,26000,870,2,152-89,2,45800-45800,26000-15000,2642.5,2642.5,3633454,91600,3989299
5,CAM006,10000,1200,PTO - EST05 - MUN11 - EST04 - MUN13 - PTO,2,0-5000-0-5000-0,10000,1200,2,119-217,2,32999-32999,9999-4999,3650.4,3650.4,5066403,65999,5497441
